## Import of External Libraries ##

In [168]:
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt

## Definition of Sorting Algorythm ##

In [170]:
def bubble_sort_by_price(df):
    # Bubble sort to sort by 'price_per_ton' in ascending order
    n = len(df)
    for i in range(n):
        for j in range(0, n-i-1):
            if df.iloc[j]['price_per_ton'] > df.iloc[j+1]['price_per_ton']:
                # Swap the rows if they are in the wrong order
                df.iloc[j], df.iloc[j+1] = df.iloc[j+1], df.iloc[j]
    return df

## Definition of Company Class ##

In [172]:
# Define the Company class with updated add_yearly_data function
class Company:
    def __init__(self, name, industry, employees):
        self.name = name # Set company name
        self.industry = industry # Set the industry, used for benchmarking
        self.employees = employees # Set the number of employees, used for benchmarking
        self.yearly_data = []  # List to store data for each year (instances of EmissionData)
        self.total_footprint = 0  # Running total of emissions
        self.total_offset = 0  # Running total of carbon offsets
        self.net_balance = 0  # Running total of footprint minus offsets
        self.credit_portfolio_df = pd.DataFrame(columns= ['project_id','name','project_type','country','duration','risk_factor','total_credit_amount','price_per_ton'])
    
    # Interactive data input functions for Scopes 1, 2, and 3
    def input_scope_1(self):
        print("\nEnter Scope 1: Direct Emissions Data")
    
        def get_positive_float(prompt):
            while True:
                try:
                    value = float(input(prompt))
                    if value < 0:
                        print("Value cannot be negative. Please enter a positive number.")
                    else:
                        return value
                except ValueError:
                    print("Invalid input. Please enter a valid number.")
    
        fuel = get_positive_float("Enter fuel consumption (liters): ")
        natural_gas = get_positive_float("Enter natural gas consumption (cubic meters): ")
        industrial_processes = get_positive_float("Enter raw material usage for industrial processes (tons): ")
        refrigerant_leakage = get_positive_float("Enter refrigerant leakage (kg): ")
    
        return {
            'fuel': fuel,
            'natural_gas': natural_gas,
            'industrial_processes': industrial_processes,
            'refrigerant_leakage': refrigerant_leakage
        }
    
    def input_scope_2(self):
        print("\nEnter Scope 2: Indirect Emissions Data")
    
        def get_positive_float(prompt):
            while True:
                try:
                    value = float(input(prompt))
                    if value < 0:
                        print("Value cannot be negative. Please enter a positive number.")
                    else:
                        return value
                except ValueError:
                    print("Invalid input. Please enter a valid number.")
    
        electricity = get_positive_float("Enter electricity usage (kWh): ")
        purchased_heat = get_positive_float("Enter purchased heat/steam usage (MWh): ")
    
        return {
            'electricity': electricity,
            'purchased_heat': purchased_heat
        }
    
    def input_scope_3(self):
        print("\nEnter Scope 3: Indirect Emissions Data")
        print("\nUpstream Emissions:")
    
        def get_positive_float(prompt):
            while True:
                try:
                    value = float(input(prompt))
                    if value < 0:
                        print("Value cannot be negative. Please enter a positive number.")
                    else:
                        return value
                except ValueError:
                    print("Invalid input. Please enter a valid number.")
    
        business_travel = get_positive_float("Enter business travel distance (km): ")
        commuting = get_positive_float("Enter average employee commuting distance per day (km): ")
        purchased_goods = get_positive_float("Enter purchased goods and services emissions (tons): ")
        waste_disposal = get_positive_float("Enter waste disposal emissions (tons): ")
    
        print("\nDownstream Emissions:")
        product_use = get_positive_float("Enter product use emissions (tons): ")
        product_end_of_life = get_positive_float("Enter product end-of-life emissions (tons): ")
        distribution_transport = get_positive_float("Enter distribution and transportation emissions (tons): ")
    
        return {
            'business_travel': business_travel,
            'commuting': commuting,
            'purchased_goods': purchased_goods,
            'waste_disposal': waste_disposal,
            'product_use': product_use,
            'product_end_of_life': product_end_of_life,
            'distribution_transport': distribution_transport
        }

    # Comprehensive function to gather data for a given year
    def add_yearly_data(self, year):
        print(f"\nAdding data for the year {year}.")
        scope_1 = self.input_scope_1()
        scope_2 = self.input_scope_2()
        scope_3 = self.input_scope_3()

        # Create a new EmissionData instance and add it to yearly data
        data = EmissionData(year, scope_1, scope_2, scope_3)
        self.yearly_data.append(data)

        # Update the overall footprint and balances
        self.total_footprint += data.total_emissions
        self.total_offset += data.total_offset # PROBABLY WRONG/ USELESS
        self.net_balance = self.total_footprint - self.total_offset # PROBABLY WRONG/ USELESS


## Definition of EmissionData Class ##

In [174]:
# Define the EmissionData class (no change from previous version)
class EmissionData:
    def __init__(self, year, scope_1, scope_2, scope_3):
        self.year = year
        self.scope_1 = scope_1
        self.scope_2 = scope_2
        self.scope_3 = scope_3
        self.total_emissions = 0
        self.total_offset = 0
        self.calculate_emissions()

    def calculate_emissions(self):
        emission_factors = {
            'fuel': 2.5,  # kg CO2 per liter of fuel
            'natural_gas': 2.0,  # kg CO2 per cubic meter
            'industrial_processes': 180,  # kg CO2 per ton
            'refrigerant_leakage': 1430,  # kg CO2 per kg of refrigerant
            'electricity': 0.4,  # kg CO2 per kWh
            'purchased_heat': 112,  # kg CO2 per MWh
            'business_travel': 0.09,  # kg CO2 per km
            'commuting': 0.05,  # kg CO2 per km
            'purchased_goods': 430,  # kg CO2 per ton
            'waste_disposal': 0.5,  # kg CO2 per ton
            'product_use': 2.0,  # kg CO2 per ton
            'product_end_of_life': 1.5,  # kg CO2 per ton
            'distribution_transport': 0.1  # kg CO2 per ton-km
        }

        scope_1_emissions = (
            self.scope_1['fuel'] * emission_factors['fuel'] +
            self.scope_1['natural_gas'] * emission_factors['natural_gas'] +
            self.scope_1['industrial_processes'] * emission_factors['industrial_processes'] +
            self.scope_1['refrigerant_leakage'] * emission_factors['refrigerant_leakage']
        )/1000 # Convert kg of CO2 into tons of CO2

        scope_2_emissions = (
            self.scope_2['electricity'] * emission_factors['electricity'] +
            self.scope_2['purchased_heat'] * emission_factors['purchased_heat']
        )/1000 # Convert kg of CO2 into tons of CO2

        scope_3_emissions = (
            self.scope_3['business_travel'] * emission_factors['business_travel'] +
            self.scope_3['commuting'] * emission_factors['commuting'] +
            self.scope_3['purchased_goods'] * emission_factors['purchased_goods'] +
            self.scope_3['waste_disposal'] * emission_factors['waste_disposal'] +
            self.scope_3['product_use'] * emission_factors['product_use'] +
            self.scope_3['product_end_of_life'] * emission_factors['product_end_of_life'] +
            self.scope_3['distribution_transport'] * emission_factors['distribution_transport']
        )/1000 # Convert kg of CO2 into tons of CO2

        self.total_emissions = scope_1_emissions + scope_2_emissions + scope_3_emissions
        

## Definition of CarbonOffest Class ##

In [176]:
# Define the CarbonOffset class that gives companies access to the Voluntary Carbon Market
class CarbonOffset:
    def __init__(self, company, carbon_projects):
        self.company = company
        self.projects_df = carbon_projects

    def display_styled_table(self, df):     
        # Define the mapping for column headers
        column_name_mapping = {
            'project_id': 'Project ID',
            'name': 'Project Name',
            'project_type': 'Project Type',
            'country': 'Country',
            'duration': 'Duration (Years)',
            'risk_factor': 'Risk Factor',
            'total_credit_amount': 'Total Credit Amount',
            'price_per_ton': 'Price per Ton (USD)'
        }
        # Rename the columns using the mapping
        df = df.rename(columns=column_name_mapping)
        
        # Remove the index column and add styles
        styled_df = df.style \
            .hide(axis="index") \
            .set_table_styles(
                [{
                    'selector': 'thead th',
                    'props': [('background-color', '#f4f4f4'), ('color', '#333'), ('font-weight', 'bold'), ('border', '1px solid black')]
                },
                {
                    'selector': 'tbody td',
                    'props': [('border', '1px solid #ddd')]
                }]
            ).set_caption("Available Carbon Offset Projects")
        
        # Display the styled DataFrame
        display(styled_df)

    def display_offset_options(self):
        # Display available offset projects and their prices with filtering options
        print("Check the available Negative Emission Projects:")
        print("We suggest, that you filter the projects to find the right credits for your high-impact carbon portfolio!")

        # Apply filters
        filtered_df = self.projects_df
        
        while True:
            print("\nYou can filter by:")
            print("1. Project Type")
            print("2. Country")
            print("3. Duration")
            print("4. Risk Factor")
            print("5. Show All")
            print("6. Continue to Purchase")
            filter_choice = input("Enter the number of the filter option (or 6 to continue): ").strip()

            if filter_choice == "1":
                while True:
                    project_type = input("Enter the project type (e.g., Biochar, Reforestation, Renewable Energy): ").strip().lower()
                    # Check if the entered project type exists in the DataFrame
                    if project_type in filtered_df['project_type'].str.lower().values:
                        filtered_df = filtered_df[filtered_df['project_type'].str.lower() == project_type]
                        break  # Exit the loop if a valid project type is entered
                    else:
                        print("Project type doesn't exist. Please enter a valid project type.")
            elif filter_choice == "2":
                while True:
                    country = input("Enter the country (e.g., Rwanda, Brazil, India): ").strip().lower()
                    # Check if the entered country exists in the DataFrame
                    if country in filtered_df['country'].str.lower().values:
                        filtered_df = filtered_df[filtered_df['country'].str.lower() == country]
                        break  # Exit the loop if a valid project type is entered
                    else:
                        print("Country doesn't exist. Please enter a valid country.")
            elif filter_choice == "3":
                while True:
                    try:
                        duration = float(input("Enter the minimum CO2 storage duration in years (e.g., 100, 1000, 10000): "))
                        if 0 <= duration:
                            break
                        else:
                            print("Duration must be a positive number. Please try again.")
                    except ValueError:
                        print("Invalid input. Please enter a valid number.")
                filtered_df = filtered_df[filtered_df['duration'] >= duration]
            elif filter_choice == "4":
                while True:
                    try:
                        risk_factor = int(input("Enter the risk factor from 1 to 5 (1 Lowest - 5 Highest): "))
                        # Check if the entered risk factor exists in the DataFrame
                        if 0 <= risk_factor <= 5:
                            break
                        else:
                            print("Risk factor must be a integer number. Please try again.")
                    except ValueError:
                        print("Invalid input. Please enter a valid number.")
                filtered_df = filtered_df[filtered_df['risk_factor'] <= risk_factor]
            elif filter_choice == "5":
                print("\nShowing all available projects:")
                filtered_df = self.projects_df
            elif filter_choice == "6":
                break
            else:
                print("Invalid choice, please try again.")

            # Display filtered options
            if filtered_df.empty:
                print("No projects match the selected filters.")
            else:
                # Sorting the filtered options based on price (ascending)
                bubble_sort_by_price(filtered_df)
                print("These are your filtered credit options filtered by price (ascending):")
                self.display_styled_table(filtered_df) # Displaying the sorted options

        # Show available projects after filtering
        print("\nAvailable Offset Projects:")
        self.display_styled_table(filtered_df)

    def purchase_offset(self, selected_id):  # Use project ID for selection
        # Find the selected project by ID
        selected_project = self.projects_df[self.projects_df['project_id'] == selected_id]
        if selected_project.empty:
            print("Project ID doesn't exists. Please try again.")
            return None
        
        # Input the desired amout of carbon credits
        while True:
            try:
                # Input the desired amount of carbon credits
                amount = int(input("Enter the amount of CO2 credits you want to purchase (tons): "))
                total_credit_amount = selected_project['total_credit_amount'].iloc[0]
                if 0 <= amount <= total_credit_amount:
                    # Reduce the total amount of credits of the selected project
                    self.projects_df.loc[self.projects_df['project_id'] == selected_id, 'total_credit_amount'] -= amount
                    break  # Exit the while loop if input is valid
                else:
                    print("The amount of CO2 credits must be between zero and the projects' total credit amount. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a integer number.")

        # Calculate total cost of the offset
        cost = amount * selected_project['price_per_ton'].iloc[0]
        selected_project_name = selected_project['name'].iloc[0]
        print(f"Purchasing {amount} tons of CO2 offset from {selected_project_name} for ${cost}.")
        print(f"The total amount of credits purchaseable from {selected_project_name} has been reduced by {amount}.")
        
        # Increase the total offset for the company
        self.company.total_offset += amount
        self.company.net_balance = self.company.total_footprint - self.company.total_offset

        # Add bought credits to the company credit portfolio
        bought_credits = selected_project.copy()
        bought_credits['total_credit_amount'] = amount
        self.company.credit_portfolio_df = pd.concat([self.company.credit_portfolio_df, bought_credits], ignore_index=True)
        
        return cost


## Definition of Report Class ##

In [178]:
# Define the parent class Report that defines emissions benchmarks and contains the child classes of more specific reports
class Report:
    def __init__(self, company):
        self.company = company

    def generate_summary(self):
        return f"Company: {self.company.name}\nIndustry: {self.company.industry}\n"
    
    def compare_to_benchmark(self, emissions_per_employee):
        industry_benchmarks = {
            'Technology': 19,  # tons CO2 per year per employee
            'Finance': 12,
            'Hospitality': 132,
            'Transportation': 378,
            'Services': 23,
            'Healthcare': 31,
            'Education': 11,
            'Retail': 37,
            'Construction': 187,
            'Manufactoring': 221,
        }

        benchmark = round(industry_benchmarks.get(self.company.industry, 10000),2) #WHERES DOES 10000 COME FROM? MUST COME FORM EMISSIONDATA
        if emissions_per_employee > benchmark:
            return f"Above benchmark by {emissions_per_employee - benchmark} tons CO2 per employee."
        else:
            return f"Below benchmark by {benchmark - emissions_per_employee} tons CO2 per employee."

    def get_selected_year(self):
        try:
            # Prompt the user to input the year
            year = int(input("Enter the year for the report: "))
            return year
        except ValueError:
            print("Invalid input. Please enter a valid year.")
            return self.get_selected_year()  # Recursive call until a valid year is entered

## Definition of Annual Report Class ##

In [180]:
# Define the child class AnnualReport that evaluates a companies annual emissions
class AnnualReport(Report):
    def __init__(self, company):
        super().__init__(company)
        self.year = None  # Year will be selected during report generation
        self.year_data = None  # Will store data for the selected year

    def generate_report(self):
        try:
            # Get the year from user input
            self.year = self.get_selected_year()
            # Find the data for the selected year
            self.year_data = next((data for data in self.company.yearly_data if data.year == self.year), None)
            
            # Handle case where no data is available for the selected year
            if not self.year_data:
                raise ValueError(f"No data available for the year {self.year}.")
            
            # Calculate emissions per employee
            emissions_per_employee = round(self.year_data.total_emissions / self.company.employees, 2)
            summary = self.generate_summary()
            
            # Generate the annual report
            report = f"{summary}Year: {self.year}\n"
            report += f"Total Emissions for {self.year}: {self.year_data.total_emissions:.2f} tons CO2\n"
            report += f"Emissions per Employee: {emissions_per_employee} tons CO2\n"
            report += self.compare_to_benchmark(emissions_per_employee)
            
            return report

        except ValueError as e:
            return str(e)
        except Exception as e:
            return f"An error occurred: {str(e)}"


## Definition of All-Time Report ##

In [182]:
# Define the child class AllTimeReport that evaluates a companies emission all-time emssions and offsets
class AllTimeReport(Report):
    def __init__(self, company):
        super().__init__(company)

    def generate_visualization(self):
        # Data for the bar chart
        labels = ['Total Footprint', 'Total Credits', 'Net Balance']
        values = [
            self.company.total_footprint,
            self.company.total_offset,
            self.company.net_balance
        ]

        # Plot the bar chart
        plt.figure(figsize=(4, 3))
        plt.bar(labels, values, color=['blue', 'green', 'orange'])
        plt.title('All-Time Emission Overview')
        plt.ylabel('Tons CO2')
        plt.xlabel('Metrics')

        # Display the plot
        plt.show()

    def generate_report(self):
        # Calculate emissions per employee
        emissions_per_employee = self.company.total_footprint / self.company.employees

        # Generate the textual summary
        summary = self.generate_summary()
        print(f"{summary}All-Time Totals:\n")
        print(f"Total Footprint: {self.company.total_footprint:.2f} tons CO2\n")
        print(self.compare_to_benchmark(emissions_per_employee))
        print(f"\nTotal Offsets: {self.company.total_offset:.2f} tons CO2\n")
        print(f"Net Carbon Balance: {self.company.net_balance:.2f} tons CO2\n\n")

        # Generate the visualization
        self.generate_visualization()


## Definition of Game Function ##

In [184]:
def main():
    print("Welcome to EcoAct!\nYour Carbon Management Partner from Footprint Assessment to Climate Action.")
    print("You are a CSO or Sustainability Manager?\nWe're going to help you assess your company's carbon footprint and directly compensate for those emissions effectively!")
    
    # Create a company
    try:
        company_name = input("\nEnter the company's name: ")

        # Validate industry inside a while loop
        allowed_industries = {'Technology', 'Finance', 'Hospitality', 'Transportation', 'Services', 'Healthcare', 'Retail', 'Education', 'Construction', 'Manufacturing'}
        while True:  # Loop until a valid industry is inputted
            industry = input("\nEnter the company's industry. Pick one of the following:\nTechnology, Finance, Hospitality, Transportation, Services, Healthcare, Retail, Education, Construction, Manufacturing\nYou picked:\n").strip()
            if industry in allowed_industries:
                break  # Exit the loop if the input is valid
            else:
                industries_string = ", ".join(allowed_industries)  # Join industry names into a single string
                print(f"Industry not recognized. Please choose from: {industries_string}.")

        # Validate employees inside a while loop
        while True:
            try:
                employees = int(input("\nEnter the company's number of employees: "))
                if employees < 1:
                    print("The number of employees cannot be zero or negative. Please enter a positive number.")
                else:
                    break  # Exit the loop if the input is valid
            except ValueError:
                print("Invalid input. Please enter a valid integer for the number of employees.")

        # Create the company object
        company = Company(company_name, industry, employees)
        print(f"Company '{company_name}' in the '{industry}' industry with {employees} employees has been created.\n")

        # Define the offset projects with inital project total credit amounts (can be reduced later)
        offset_projects = [
            {'project_id': 'VCS1001', 'name': 'UNDO Biochar Rwanda', 'project_type': 'Biochar', 'country': 'Rwanda', 'duration': 1000, 'risk_factor': 2, 'total_credit_amount': 5000, 'price_per_ton': 214.56},
            {'project_id': 'VCS1002', 'name': 'Ginco Biochar New Dehli', 'project_type': 'Biochar', 'country': 'India', 'duration': 1000, 'risk_factor': 2, 'total_credit_amount': 4420, 'price_per_ton': 245.67},
            {'project_id': 'VCS1003', 'name': 'Green Leaf Biochar Brazil', 'project_type': 'Biochar', 'country': 'Brazil', 'duration': 1500, 'risk_factor': 1, 'total_credit_amount': 7000, 'price_per_ton': 221.34},
            {'project_id': 'VCS1004', 'name': 'Eco Future Biochar Kenya', 'project_type': 'Biochar', 'country': 'Kenya', 'duration': 2000, 'risk_factor': 1, 'total_credit_amount': 5500, 'price_per_ton': 200.45},
            {'project_id': 'VCS1005', 'name': 'Sunshine Biochar Indonesia', 'project_type': 'Biochar', 'country': 'Indonesia', 'duration': 1800, 'risk_factor': 2, 'total_credit_amount': 4800, 'price_per_ton': 234.89},
            {'project_id': 'VCS1006', 'name': 'Solar Green Reforestation Brazil', 'project_type': 'Reforestation', 'country': 'Brazil', 'duration': 250, 'risk_factor': 3, 'total_credit_amount': 10000, 'price_per_ton': 15.34},
            {'project_id': 'VCS1007', 'name': 'Evergreen Reforestation Rwanda', 'project_type': 'Reforestation', 'country': 'Rwanda', 'duration': 120, 'risk_factor': 5, 'total_credit_amount': 4000, 'price_per_ton': 18.75},
            {'project_id': 'VCS1008', 'name': 'Wildlife Reforestation Mexico', 'project_type': 'Reforestation', 'country': 'Mexico', 'duration': 180, 'risk_factor': 4, 'total_credit_amount': 8000, 'price_per_ton': 16.45},
            {'project_id': 'VCS1009', 'name': 'Mountain View Reforestation India', 'project_type': 'Reforestation', 'country': 'India', 'duration': 200, 'risk_factor': 4, 'total_credit_amount': 15000, 'price_per_ton': 14.76},
            {'project_id': 'VCS1010', 'name': 'Ocean Breeze Reforestation Indonesia', 'project_type': 'Reforestation', 'country': 'Indonesia', 'duration': 150, 'risk_factor': 4, 'total_credit_amount': 5000, 'price_per_ton': 11.89},
            {'project_id': 'VCS1011', 'name': 'Solar Eco Renewable Energy USA', 'project_type': 'Renewable Energy', 'country': 'USA', 'duration': 0, 'risk_factor': 8, 'total_credit_amount': 12000, 'price_per_ton': 22.45},
            {'project_id': 'VCS1012', 'name': 'Wind Power Renewable Energy India', 'project_type': 'Renewable Energy', 'country': 'India', 'duration': 0, 'risk_factor': 7, 'total_credit_amount': 8000, 'price_per_ton': 18.92},
            {'project_id': 'VCS1013', 'name': 'SolarWind Renewable Energy Kenya', 'project_type': 'Renewable Energy', 'country': 'Kenya', 'duration': 0, 'risk_factor': 8, 'total_credit_amount': 3000, 'price_per_ton': 12.34},
            {'project_id': 'VCS1014', 'name': 'Green Horizon Renewable Energy Brazil', 'project_type': 'Renewable Energy', 'country': 'Brazil', 'duration': 0, 'risk_factor': 7, 'total_credit_amount': 6000, 'price_per_ton': 19.45},
            {'project_id': 'VCS1015', 'name': 'Blue Sky Renewable Energy Indonesia', 'project_type': 'Renewable Energy', 'country': 'Indonesia', 'duration': 0, 'risk_factor': 8, 'total_credit_amount': 4500, 'price_per_ton': 17.62},
            {'project_id': 'VCS1016', 'name': 'Solar Future Renewable Energy Mexico', 'project_type': 'Renewable Energy', 'country': 'Mexico', 'duration': 0, 'risk_factor': 9, 'total_credit_amount': 7000, 'price_per_ton': 21.33},
            {'project_id': 'VCS1017', 'name': 'Wind Valley Renewable Energy South Africa', 'project_type': 'Renewable Energy', 'country': 'South Africa', 'duration': 0, 'risk_factor': 8, 'total_credit_amount': 11000, 'price_per_ton': 14.87},
            {'project_id': 'VCS1018', 'name': 'Ocean Renewable Energy Australia', 'project_type': 'Renewable Energy', 'country': 'Australia', 'duration': 0, 'risk_factor': 8, 'total_credit_amount': 15000, 'price_per_ton': 25.00},
            {'project_id': 'VCS1019', 'name': 'Mangrove Restoration Indonesia', 'project_type': 'Restoration', 'country': 'Indonesia', 'duration': 250, 'risk_factor': 7, 'total_credit_amount': 5000, 'price_per_ton': 18.45},
            {'project_id': 'VCS1020', 'name': 'Forest Regrowth Kenya', 'project_type': 'Restoration', 'country': 'Kenya', 'duration': 150, 'risk_factor': 6, 'total_credit_amount': 8000, 'price_per_ton': 16.78},
            {'project_id': 'VCS1021', 'name': 'Wetland Recovery USA', 'project_type': 'Restoration', 'country': 'USA', 'duration': 200, 'risk_factor': 5, 'total_credit_amount': 7000, 'price_per_ton': 21.23},
            {'project_id': 'VCS1022', 'name': 'Soil Carbon Sequestration South Africa', 'project_type': 'Soil Carbon Sequestration', 'country': 'South Africa', 'duration': 10000, 'risk_factor': 1, 'total_credit_amount': 4500, 'price_per_ton': 28.99},
            {'project_id': 'VCS1023', 'name': 'Agroforestry Restoration Brazil', 'project_type': 'Restoration', 'country': 'Brazil', 'duration': 250, 'risk_factor': 5, 'total_credit_amount': 6500, 'price_per_ton': 18.11},
            {'project_id': 'VCS1024', 'name': 'Carbon Sequestration Australia', 'project_type': 'Soil Carbon Sequestration', 'country': 'Australia', 'duration': 12000, 'risk_factor': 3, 'total_credit_amount': 5000, 'price_per_ton': 27.88},
            {'project_id': 'VCS1025', 'name': 'Mountain Carbon Sequestration USA', 'project_type': 'Soil Carbon Sequestration', 'country': 'USA', 'duration': 15000, 'risk_factor': 2, 'total_credit_amount': 4000, 'price_per_ton': 26.55},
            {'project_id': 'VCS1026', 'name': 'Savanna Sequestration Kenya', 'project_type': 'Soil Carbon Sequestration', 'country': 'Kenya', 'duration': 20000, 'risk_factor': 2, 'total_credit_amount': 3000, 'price_per_ton': 25.45},
            {'project_id': 'VCS1027', 'name': 'Desert Afforestation South Africa', 'project_type': 'Afforestation', 'country': 'South Africa', 'duration': 300, 'risk_factor': 8, 'total_credit_amount': 15000, 'price_per_ton': 21.77},
            {'project_id': 'VCS1028', 'name': 'Urban Tree Planting India', 'project_type': 'Afforestation', 'country': 'India', 'duration': 150, 'risk_factor': 7, 'total_credit_amount': 5000, 'price_per_ton': 12.12},
            {'project_id': 'VCS1029', 'name': 'Green World Afforestation Brazil', 'project_type': 'Afforestation', 'country': 'Brazil', 'duration': 120, 'risk_factor': 6, 'total_credit_amount': 4000, 'price_per_ton': 10.56},
            {'project_id': 'VCS1030', 'name': 'Redwoods Afforestation USA', 'project_type': 'Afforestation', 'country': 'USA', 'duration': 200, 'risk_factor': 7, 'total_credit_amount': 6000, 'price_per_ton': 8.89},
            {'project_id': 'VCS1031', 'name': 'Rainforest Revival Indonesia', 'project_type': 'Afforestation', 'country': 'Indonesia', 'duration': 180, 'risk_factor': 8, 'total_credit_amount': 4500, 'price_per_ton': 9.34},
            {'project_id': 'VCS1032', 'name': 'Amazonian Reforestation Brazil', 'project_type': 'Reforestation', 'country': 'Brazil', 'duration': 250, 'risk_factor': 7, 'total_credit_amount': 8000, 'price_per_ton': 17.50}
        ]

        # Convert the offset projects into a pandas DataFrame
        offset_projects_df = pd.DataFrame(offset_projects)
        projects_df = offset_projects_df.copy()
    
    except ValueError as e:
        print(f"Error: {e}")
        return  # Exit the program if input is invalid
        

    while True:
        print("\nWhat would you like to do?")
        print("1. Add your yearly emission data")
        print("2. View all-time report")
        print("3. View yearly report")
        print("4. Purchase carbon offsets")
        print("5. View your carbon portfolio")
        print("6. Exit")
        choice = input("Enter your choice (number 1-6): ")
        
        if choice == "1":
            # Add yearly emission data
            try:
                year = int(input("\nEnter the year for which you are adding data: "))
                company.add_yearly_data(year)
                print(f"Yearly data for {year} has been added successfully.\n")
            except ValueError:
                print("Invalid year. Please enter a valid number.")

        elif choice == "2":
            # Generate and display all-time report
            print("\nAll-Time Report:")
            all_time_report = AllTimeReport(company)
            print(all_time_report.generate_report())

        elif choice == "3":
            # Generate and display annual report
            print("\nYearly Report:")
            annual_report = AnnualReport(company)
            print(annual_report.generate_report())

        elif choice == "4":
            # Purchase carbon offsets
            try:
                print("\nPurchasing Carbon Offsets:")
                offset = CarbonOffset(company, projects_df)
                offset.display_offset_options()
                while True:
                    try:
                        # Prompt the user to input a project ID
                        project_id = input("Choose the project ID: ")
                        
                        # Check if the project ID exists in the DataFrame
                        if project_id in offset.projects_df['project_id'].astype(str).values:
                            # If valid, break the loop
                            break
                        else:
                            print("Project ID does not exist. Please enter a valid Project ID.")
                    except Exception as e:
                        print(f"An error occurred: {e}. Please try again.")
                # Proceed with the purchase
                cost = offset.purchase_offset(project_id)
                print(f"Offset purchased successfully. Total cost: ${cost:.2f}\n")
            except ValueError:
                print("Invalid input. Please enter a valid input.")
        
        elif choice == "5":
            # View carbon portfolio
            print("\nViewing Carbon Portfolio:")
            if company.credit_portfolio_df.empty:
                print("Your carbon portfolio is empty. Purchase offsets to see them here.")
            else:
                offset.display_styled_table(company.credit_portfolio_df)

        elif choice == "6":
            # Exit the program
            print("\nThank you for using EcoAct.\nSee you next year to compensate your emissions again and make this world a better place!")
            break

        else:
            print("\nInvalid choice. Please try again.")

In [ ]:
main()

Welcome to EcoAct!
Your Carbon Management Partner from Footprint Assessment to Climate Action.
You are a CSO or Sustainability Manager?
We're going to help you assess your company's carbon footprint and directly compensate for those emissions effectively!
